In [ ]:
from netgen.occ import *
from ngsolve import *

"""
James Elgy - 2022:
sphere (with prismatic boundary layers) example for Netgen OCC geometry mesh generation.
"""



# Setting mur, sigma, and defining the top level object name:
material_name = ['mat1']
mur = [32]
sigma = [1e6]
# We can also set the object size alpha here.
alpha = 0.01

# Setting Boundary layer Options:
max_target_frequency = 1e8
boundary_layer_material = material_name[0]
number_of_layers = 2

# setting radius
r = 1

# Generating OCC primative sphere centered at [0,0,0] with radius r:
sphere = Sphere(Pnt(0,0,0), r=r)

# Here, we split the sphere into positive and negative hemispheres. This is to provide a surface for the prismatic layers.
pos_sphere = sphere - Box(Pnt(0,100,100), Pnt(-100,-100,-100))
neg_sphere = sphere - Box(Pnt(0,100,100), Pnt(100,-100,-100))
sphere = pos_sphere + neg_sphere

# Generating surrounding non-conducting region as [-1000,1000]^3 box:
box = Box(Pnt(-1000, -1000, -1000), Pnt(1000,1000,1000))

# setting material and bc names:
# For compatability, we want the non-conducting region to have the 'outer' boundary condition and be labeled as 'air'
sphere.mat(material_name[0])
sphere.bc('default')
box.mat('air')
box.bc('outer')

# Setting maxh:
sphere.maxh = 0.4
box.maxh = 1000

# Joining the two meshes:
# Glue joins two OCC objects together without interior elemements
joined_object = Glue([sphere, box])

# Generating Mesh:
geo = OCCGeometry(joined_object)
nmesh = geo.GenerateMesh()

# Applying Boundary Layers:
mu0 = 4*3.14159 * 1e-7
tau = (2/(max_target_frequency * sigma[0] * mu0 * mur[0]))**0.5 / alpha
layer_thicknesses = [(2**n)*tau for n in range(number_of_layers)]

nmesh.BoundaryLayer(boundary=".*", thickness=layer_thicknesses, material=boundary_layer_material,
                           domains=boundary_layer_material, outside=False)


nmesh.Save(r'VolFiles/OCC_Template_Prisms.vol')



